# Diseño del controlador

Como se indicó en el apartado anterior, las ecuaciones de nuestro sistema tienen la siguiente forma:

$$
\begin{align}
    \dot{\mathbf x} &= \mathbf A\mathbf x + \mathbf Bu \\
    \mathbf y &= \mathbf C\mathbf x + \mathbf Du
\end{align}
$$

En la bibliografía sobre Control es frecuente referirse al sistema que queremos controlar, de modo general, como *planta*. Cuando no hay realimentación, el sistema se conoce como *planta en lazo abierto*. Si consideramos el sistema libre, sin introducir ningún *input*, resultará una ecuación diferencial de la forma:

$$
    \dot{\mathbf x} = \mathbf A\mathbf x
$$

Se dice que un sistema es *asintóticamente estable* cuando, para unas condiciones iniciales cualesquiera, el vector de estados $\mathbf x$ tiende a cero cuando el tiempo tiende a infinito. Se puede demostrar que la solución de un sistema de este tipo es estable cuando todos los autovalores de $\mathbf A$ tienen parte real negativa. Como se puede ver, nuestro sistema no es estable, ya que tiene un autovalor positivo:

In [ ]:
import numpy as np

# Cargamos las matrices A, B, C, y D del paso anterior
# Aprovechamos para cargar las dimensiones del modelo, para las animaciones
with np.load('./Data/Model.npz') as m:
    A, B, C, D = (m['A'], m['B'], m['C'], m['D'])
    L, R = (5*m['r'], m['r'])
    
# Comprobamos los autovalores de 'A'
display(np.real(np.linalg.eig(A)[0]))

## Estabilización mediante realimentación de estados

La realimentación de estados se basa en definir la entrada del sistema $u$ como una realimentación negativa del estado, de la siguiente forma:

$$
    u = -\mathbf K\mathbf x
$$

Con este tipo de realimentación, se puede determinar de forma analítica una matriz de ganancias $\mathbf K$ que estabilice el sistema, es decir, que haga que todos los autovalores sean negativos. Para verlo más claramente, se puede introducir la realimentación en las ecuaciones del sistema:

$$
    \dot{\mathbf x} = \mathbf{Ax} - \mathbf{BKx} = \left(\mathbf A - \mathbf{BK}\right)\mathbf x
$$

Se observa que, ahora, la estabilidad del sistema la determinan los autovalores de $\mathbf A - \mathbf{BK}$. Esto significa que se puede alterar la dinámica del sistema a conveniencia, ajustando la matriz $\mathbf K$ para colocar los autovalores donde nos interese. Este método de control se puede esquematizar de la forma siguiente:

![Diagrama de bloques](./Images/figure-01.png)

En la figura se muestra un sistema con una entrada y una salida, es decir, un sistema SISO (*Single Input, Single Output*), pero todo lo aquí desarrollado se puede generalizar fácilemente a sistemas con varias entradas y salidas. Por ahora no vamos a considerar la entrada de referencia $r$.

## Regulador cuadrático lineal (LQR)

El esquema de control de realimentación de estados deja abierta una decisión complicada al diseñador: ¿dónde colocar los autovalores del sistema? Existen diversos métodos descritos en la bibliografía, dedicados a determinar dónde se deben colocar los autovalores para conseguir que la respuesta del sistema presente unas características determinadas. Estas técnicas se denominan *asignación de polos* o *pole placement*.

En nuestro proyecto, vamos a utilizar un método un poco más elaborado, pero a la vez sencillo de utilizar: el LQR (*Linear Quadratic Regulator*). El LQR no es más que otro método para calcular la matriz $\mathbf K$, pero en lugar de tener que decidir dónde colocamos los polos del sistema, lo que hacemos es buscar el valor único de $\mathbf K$ que minimiza la siguiente función:

$$
    J = \int_0^\infty\left(\mathbf x^T\mathbf Q\mathbf x + \mathbf u^T\mathbf R\mathbf u\right)dt
$$

Esta función (expresada en forma general, para un sistema con múltiples *inupts*) combina los valores cuadráticos, integrados a lo largo del tiempo, de las dos magnitudes que queremos minimizar: el error en los estados y la magnitud de las entradas de control. Cada término va ponderado por una matriz de pesos, $\mathbf Q$ para los estados y $\mathbf R$ para los *inputs*, de modo que podemos decidir qué variables queremos penalizar más en el controlador. Por ejemplo, asignar un valor elevado al elemento $Q_{ii}$ de la diagonal de $\mathbf Q$ hará que los errores en el estado $x_i$ sean fuertemente penalizados en la función objetivo $J$, así que el controlador se esforzará en manternerlos bajos. Por el contrario, si se hace lo mismo con el elemento $R_{jj}$ de $\mathbf R$, se indica al controlador que debe utilizar lo menos posible el actuador correspondiente al *input* $u_j$.

Modificando los elementos de las matrices $\mathbf Q$ y $\mathbf R$ se puede ajustar el comportamiento del controlador a nuestros requerimientos, dando mayor relevancia al error en los estados o al esfuerzo de control. Lo primero puede ser importante si en algún estado es crítico mantener errores bajos, y lo segundo, por ejemplo, si es prioritario obtener una alta eficiencia energética, aunque sea al coste de menor precisión en el control.

Una vez determinadas las matrices de pesos, la función `lqr` del módulo de control de Python resuelve el problema de optimización y nos devuelve directamente la matriz de ganancias $\mathbf K$ que minimiza la función $J$:

In [ ]:
from control import ss, lqr, ss2tf, minreal, forced_response

# Construimos un sistema state-space con las matrices A, B, C y D
sys = ss(A, B, C, D)

Qc = np.eye(3) # Aproximación incial para 'Q'
Qc[2, 2] = 200 # Ajustamos peso de la velociad 'xp'
Rc = 1         # Peso del voltaje 'V'

K = lqr(sys, Qc, Rc)[0]

Para evaluar el comportamiento del regulador, se simula el comportamiento del sistema cuando partimos de una posición fuera de equilibrio, con un ángulo incial $\theta_0$ de 30º. La entrada de referencia $r$ que se ve en la figura se dejará a cero (se explicará más adelante para qué sirve esta entrada). Como ejercicio, es interesante comprobar cómo varía la repuesta al modificar los pesos del LQR:

In [ ]:
%%capture
from matplotlib.animation import FuncAnimation
import plotfunctions as pf

In [ ]:
# Ensamblamos el sistema en lazo cerrado
clsys = ss(A - B*K, B, C, D)

tf = 3.00         # Duración de la simulación en segundos
dt = 0.01         # Paso de tiempo de simulación en segundos

# Simulamos respuesta para ángulo incial de 30 grados
t = np.arange(0, tf, dt)
r = np.zeros(t.shape)

T, Y, X = forced_response(clsys, t, r, X0=[np.radians(30), 0, 0])

pf.plot_response(T, X)

Como se ve, el sistema es estable, convergiendo rápidamente tanto $\theta$ como $\dot x$ a cero.

Podemos verificar que, al aplicar la realimentación, todos los autovalores del sistema tienen ahora parte real negativa:

In [ ]:
display(np.real(np.linalg.eig(A - B*K)[0]))

A continuación se muestra una animación con el movimiento resultante:

In [ ]:
FuncAnimation(pf.fg1, pf.anim, init_func=pf.init, frames=len(T), fargs=(X[[0, 2]], dt, R, L), interval=dt*1000, blit=True)

# Control de la velocidad

Cuando se aplica realimentación de estados, el controlador resultante siempre tratará de llevarlos todos al equilibrio, y eso es lo que se conoce como *regulador*. Hasta ahora, esto nos ha servido para estabilizar el sistema, manteniéndolo equilibrado en vertical y con velocidad cero. Si lo que queremos es asignar a algún *output* un valor arbitrario, hay que convertir nuestro regulador en un *servomecanismo*. El objetivo ahora es conseguir que alguno de los *outputs* (en nuestro caso la velocidad $\dot x$) siga a un valor de referencia $r$, en lugar de converger hacia cero.

Antes de explicar cómo se consigue esto, es importante introducir un nuevo concepto: el *tipo* de sistema o planta. Por definición, una planta es de tipo $N$ cuando, al representarla como un sistema con realimentación unitaria (como el que se muestra en la figura), el término de grado más bajo del denominador de la función de transferencia $G$ es de grado $N$.

![Realimentación unitaria](./Images/figure-04.png)

Es decir, si la función de transferencia en lazo abierto $G$ se escribe en la forma:

$$
    G(s) = \frac{K\left(T_as + 1\right)\left(T_bs + 1\right)\cdots\left(T_ms + 1\right)}
    {s^N\left(T_1s + 1\right)\left(T_2s + 1\right)\cdots\left(T_ps + 1\right)}
$$

el tipo de sistema corresponderá al exponente $N$ que aparece en el denominador. El tipo de un sistema determina su error de seguimiento $e$ en régimen permanente para distintos tipos de entrada. Por ejemplo, en un sistema tipo 1, el error tenderá a cero para una entrada escalón, a un valor finito para una entrada rampa, y a infinito para una entrada cuadrática o de orden superior. En cambio, un sistema tipo 0 ya mostrará un error finito en régimen permanente para una entrada escalón, y será inestable frente a una entrada rampa o superior.

Para saber el tipo de nuestro sistema, primero tenemos que representarlo según la estructura de la figura. La función de transferencia $H$ del sistema completo, entre la entrada $r$ y la salida $y$, se puede obtener sustituyendo el error de seguimiento $e$ por su valor $r - y$:

$$
    y = G\left(r - y\right) \implies H(s) = \frac{y}{r} = \frac{G}{1 + G}
$$

Entonces, se puede hacer el cambio inverso para expresar cualquier sistema con función de transferencia $H$ como sistema con realimentación unitaria:

$$
    G(s) = \frac{H}{1 - H}
$$

Comprobamos el tipo de nuestro sistema, incluyendo la realimentación, cuando consideramos que nuestro único *output* es la velocidad. La función `ss2tf` devuelve la función de transferencia $H$ de un sistema *state-space*, y `minreal` (*minimal realization*) cancela los factores comunes del numerador y el denominador:

In [ ]:
# Matrices de salida si el output es sólo la velocidad
Cc = C[1:]
Dc = D[1:]

# Función de transferencia del sistema con realimentación
H = ss2tf(ss(A - B*K, B, Cc, Dc))

# Calculamos G para transformar en sistema con realimentación unitaria
minreal(H/(1 - H))

Mirando el denominador, se ve claramente que nuestro sistema realimentado es de tipo 0. Eso significa que, para una entrada escalón, la salida $\dot x$ nunca alcanzará el valor de referencia $r$ en régimen permanente. Si queremos controlar la velocidad correctamente, tendremos que convertir el sistema en uno de tipo 1.

## Servomecanismo tipo 1 para controlar la velocidad

Existen varios métodos para convertir un sistema de tipo 0 en uno de tipo 1. Lo más sencillo es escalar la entrada de referencia $r$ mediante una ganancia de precompensación $\bar N$, que haga que el *output* que queremos controlar (en nuestro caso la velocidad $\dot x$) alcance el valor solicitado en régimen permanente. En la figura se muestra un diagrama del sistema original, con la precompensación aplicada en la entrada:

![Diagrama con precompensación](./Images/figure-02.png)

Al añadir la precompensación, la entrada del sistema en lazo abierto pasa a ser:

$$
    u = -\mathbf{Kx} + \bar Nr
$$

Vamos a estudiar lo que ocurre con $\mathbf x$ y $u$ cuando tenemos una entrada de referencia $r$ de tipo escalón unitario. Supondremos que, en régimen permanente, la salida $y$ alcanza efectivamente el valor de referencia (es decir, la unidad). Como en régimen estacionario las derivadas de los estados $\dot{\mathbf x}$ serán nulas, podemos escribir:

$$
    \begin{bmatrix} \mathbf 0 \\ 1 \end{bmatrix} = 
    \begin{bmatrix} \mathbf A && \mathbf B \\ \mathbf C && D \end{bmatrix}
    \begin{bmatrix} \mathbf{x_\infty} \\ u_\infty \end{bmatrix}
$$

Si la matriz de este sistema lineal es invertible, resolviéndolo obtendremos el valor de $\mathbf x_\infty$ y $u_\infty$. Como la referencia $r$ es un escalón unitario, la entrada del sistema en lazo abierto alcanzará el siguiente valor:

$$
    u_\infty = -\mathbf{Kx}_\infty + \bar N
$$

de donde obtenemos $\bar N$ como:

$$
    \bar N = \mathbf{Kx}_\infty + u_\infty
$$

In [ ]:
# Precompensación para salida 'xp' en régimen permanente
xuinf = np.linalg.solve(np.block([[A, B], [Cc, Dc]]), [0, 0, 0, 1])
Nb = K@xuinf[:-1] + xuinf[-1]

Antes de simular la respuesta, podemos montar el sistema en lazo cerrado con precompensación en la entrada, y comprobar su tipo:

In [ ]:
# Sistema en lazo cerrado, incluyendo la precompensación
clsys = ss(A - B*K, B*Nb, Cc, Dc)

# Función de transferencia del sistema con realimentación
H = ss2tf(clsys)

# Calculamos G(s) para sistema con realimentación unitaria
minreal(H/(1 - H))

Vemos que el sistema se puede considerar prácticamente de tipo 1, ya que el término de grado 0 del denominador es muy pequeño.

Para evaluar el comportamiento del controlador, se simulará su respuesta a una entrada escalón de 0.3 m/s seguida de una parada:

In [ ]:
# Simulamos respuesta a escalón de 0.3 m/s con parada
t = np.arange(0, tf, dt)
r = np.zeros(len(t))
r[t < 1.5] = 0.3

T, Y, X = forced_response(clsys, t, r)

pf.plot_response(T, X)

Se observa que el sistema alcanza efectivamente la velocidad objetivo y luego se para, manteniendo siempre la barra en equilibrio.

Si este controlador se va a implementar en el robot real, hay que verificar que el voltaje no supere los 6V:

In [ ]:
pf.plot_voltage(T, (Nb*r - K@X).T)

Vemos que el sistema puede realizar la maniobra solicitada sin saturar el motor. En caso contrario, habría que variar los pesos del LQR, o realizar una maniobra más suave. A continuación se muestra una animación del movimiento obtenido:

In [ ]:
FuncAnimation(pf.fg1, pf.anim, init_func=pf.init, frames=len(T), fargs=(X[[0, 2]], dt, R, L), interval=dt*1000, blit=True)

Los parámetros obtenidos ya sirven para controlar el robot en velocidad. Los valores mostrados más abajo se pueden copiar y pegar directamente en un *sketch* de Arduino, de forma que, para implementar el controlador, sólo habría que calcular el voltaje como $V = -\mathbf{Kx} + \bar Nr$.

In [ ]:
print('const float K1 =%10.6f;  // Ganancia theta'    % K[0, 0])
print('const float K2 =%10.6f;  // Ganancia thetap'   % K[0, 1])
print('const float K3 =%10.6f;  // Ganancia xp'       % K[0, 2])
print('const float Nb =%10.6f;  // Precompensación'   % Nb)

Una de las principales desventajas de los métodos de realimentación de estados es que, para calcular $u$, necesitamos conocer el valor de todos los estados. Esto es un problema, porque normalmente no todos los estados se pueden medir fácilmente con precisión. En estos casos, lo que se hace es utilizar un *observador*, que consiste en un modelo del sistema que se va integrando en el tiempo, corrigiendo los errores de integración a partir de los datos de los sensores.

En este proyecto hay dos estados, $\dot\theta$ y $\dot x$, que son muy fáciles de medir, ya que se obtienen a partir del giróscopo y los *encoders*, pero el ángulo $\theta$ no se puede medir directamente con los sensores que tenemos. Como el uso de observadores embebidos en el controlador queda fuera del ámbito de este proyecto, estimaremos $\theta$ utilizando un filtro de Kalman simplificado, como se explica en el [documento correspondiente](./Kalman.ipynb).

## Servomecanismo tipo 1 con integrador en la entrada

El problema de usar precompensación es que la respuesta es muy sensible a errores en los parámetros del sistema. Cualquier desviación en el valor de $\bar N$ hará que el sistema vuelva a ser de tipo 0, y por lo tanto tenga error en régimen permanente. La manera de resolver esto de forma más robusta es añadir a la entrada un integrador, en lugar de una simple ganancia:

![Diagrama con integral](./Images/figure-03.png)

Para utilizar este esquema de control, hay que aumentar el sistema añadiéndole un estado adicional $\xi$, que será la integral del error de seguimiento $r - y$. Por lo tanto, su derivada es:

$$
    \dot\xi = r - y = r - \left(\mathbf{Cx} + Du\right)
$$

y la entrada $u$ pasa a ser ahora:

$$
    u = -\mathbf K \mathbf x + k_I\xi
$$

La ecuación completa de la dinámica del sistema aumentado, sin incluir la realimentación, se puede escribir como:

$$
    \begin{bmatrix} \dot{\mathbf x} \\ \dot\xi \end{bmatrix} = 
    \begin{bmatrix} \mathbf A && \mathbf 0 \\ -\mathbf C && 0 \end{bmatrix}
    \begin{bmatrix} \mathbf{x} \\ \xi\end{bmatrix} + 
    \begin{bmatrix} \mathbf B \\ -D \end{bmatrix}u + 
    \begin{bmatrix} \mathbf 0 \\ 1 \end{bmatrix}r
$$

Estudiaremos la respuesta de este sistema a una entrada escalón, cuando el tiempo tiende a infinito. Para ello, definiremos las siguientes magnitudes:

$$
    \mathbf e = \begin{bmatrix} \mathbf{x} - \mathbf{x}_\infty \\ \xi - \xi_\infty \end{bmatrix} \qquad
    u_e = u - u_\infty
$$

que representan las desviaciones del estado y la entrada respecto sus propios valores en régimen permanente. Como $r$ es un escalón, su valor es siempre el mismo para $t>0$, de forma que podemos escribir la dinámica del error como:

$$
    \dot{\mathbf e} = 
    \begin{bmatrix} \mathbf A && \mathbf 0 \\ -\mathbf C && 0 \end{bmatrix}
    \mathbf e + 
    \begin{bmatrix} \mathbf B \\ -D \end{bmatrix}u_e = \hat{\mathbf A}\mathbf e + \hat{\mathbf B}u_e
$$

Se puede observar que, de forma análoga a lo que ocurría al principio, tenemos un sistema en forma *state-space* cuyos estados tienen que converger asintóticamente a cero. Eso quiere decir que existirán unas ganancias de realimentación $\hat{\mathbf K}$ que estabilicen el sistema:

$$
    \hat{\mathbf K} = \begin{bmatrix} \mathbf K && -k_I \end{bmatrix}
$$

Estas ganancias se pueden calcular, como antes, utilizando el algoritmo LQR, sabiendo que ahora tenemos cuatro estados, ya que el último corresponde a la integral del error de seguimiento en la velocidad:

In [ ]:
# Ecuaciones del sistema con los errores como estados
# Se añade la integral del error en velocidad como cuarto estado
Ah = np.block([[A, np.zeros((3, 1))], [-Cc, 0]])
Bh = np.block([[B], [-Dc]])

# Controlador LQR para la dinámica del error
Qc = np.eye(4)   # Aproximación inicial para 'Q'
Qc[0, 0] = 0.1   # Ajustamos peso del ángulo 'th'
Qc[1, 1] = 0.01  # Ajustamos peso de la derivada del ángulo 'thp'
Qc[2, 2] = 2     # Ajustamos peso de la velocidad 'xp'
Qc[3, 3] = 500   # Peso de la integral del error en velocidad 'xi'
Rc = 1           # Peso del voltaje 'V'

# La función 'LQR' también admite usar sólo las matrices A y B
Kh = lqr(Ah, Bh, Qc, Rc)[0]
Kn = Kh[0, :-1]
Ki = -Kh[0, -1]

Para simular el sistema, utilizaremos un modelo aumentado de la siguiente forma, que se obtiene al sustituir en la ecuación del sistema el valor de $u$:

$$
    \begin{bmatrix} \dot{\mathbf x} \\ \dot\xi \end{bmatrix} = 
    \begin{bmatrix} \mathbf A - \mathbf{BK} && \mathbf Bk_I \\ D\mathbf K - \mathbf C && -Dk_I \end{bmatrix}
    \begin{bmatrix} \mathbf{x} \\ \xi\end{bmatrix} + 
    \begin{bmatrix} \mathbf 0 \\ 1 \end{bmatrix}r
$$

Antes de simular, verificamos que este sistema es de tipo 1:

In [ ]:
# Sistema en lazo cerrado con las ganancias obtenidas
An = np.block([[A - B*Kn, B*Ki], [Dc*Kn - Cc, -Dc*Ki]])
Bn = [[0], [0], [0], [1]]
Cn = [0, 0, 1, 0]
Dn = 0

clisys = ss(An, Bn, Cn, Dn)

# Función de transferencia del sistema con realimentación
H = ss2tf(clisys)

# Calculamos G(s) para sistema con realimentación unitaria
minreal(H/(1 - H))

Para comprobar que este método es mucho más robusto que la precompensación, se puede hacer una prueba sencilla. En el modelo con precompensación, si antes de ensamblar el sistema *state-space* se simula un error en los datos del modelo multiplicando $\bar N$ por un factor, por ejemplo 0.9, se verá que el sistema pasa a ser tipo 0, incluso con valores del factor muy cercanos a la unidad. En cambio, si se hace lo mismo con $k_I$, se observa que el término de grado 0 del denominador se mantiene cercano a cero para cualquier valor de $k_I$.

A continuación se muestra gráficamente la respuesta, comparándola con la obtenida usando precompensación (en línea de puntos). Igual que antes, es interesante probar diferentes pesos en el LQR, para ver cómo varía el comportamiento del sistema.

In [ ]:
# Respuesta con misma entrada que en el modelo con precompensación
T, Y, Xn = forced_response(clisys, t, r)

pf.plot_response(T, Xn, X)

Antes de implementar el controlador, hay que asegurarse de que no estamos saturando el actuador:

In [ ]:
pf.plot_voltage(T, (Ki*Xn[3, :] - Kn@Xn[:3, :]).T)

Vemos que los voltajes se mantienen por debajo de los 6V. Por último, visualizamos el movimiento:

In [ ]:
FuncAnimation(pf.fg1, pf.anim, init_func=pf.init, frames=len(T), fargs=(Xn[[0, 2]], dt, R, L), interval=dt*1000, blit=True)

Como todo es correcto, ya podemos utilizar las ganancias para la implementación física del controlador. Se pueden copiar y pegar directamente en un sketch de Arduino, siendo ahora el voltaje $V= -\mathbf{Kx} + k_I\xi$. Esto significa que, además de los estados anteriores, habrá que ir calculando la integral del error en velocidad.

In [ ]:
print('const float K1 =%10.6f;  // Ganancia theta'    % Kn[0, 0])
print('const float K2 =%10.6f;  // Ganancia thetap'   % Kn[0, 1])
print('const float K3 =%10.6f;  // Ganancia xp'       % Kn[0, 2])
print('const float Ki =%10.6f;  // Ganancia integral' % Ki)